# Sentence Generator

In [22]:
from random import choice, random
from ipywidgets import Output, Button

def coin():
    """
    Return a random boolean
    """
    return random() > 0.5

def display_helper(func):
    """
    A helper function which takes a sentence generator
    and displays a widget that lets you call the generator
    repeatedly by clicking a button
    
    :param func: sentence generator function
    """
    genbutton = Button(description='Generate')
    output = Output()
    display(genbutton, output)

    def generate(event):
        sentence = func()
        output.clear_output()
        with output:
            print(sentence)

    genbutton.on_click(generate)
    generate(None)
    return func

Start with simple sentence that takes a noun a verb and another noun

In [27]:
noun_a = [
    'John',
    'Jim',
    'Gordon Ramsay',
    'Steven',
    'Khan',
    'Roy',
    'Michael',
    'Vladmir Putin'
]

verb = [
    'jumps',
    'runs',
    'cooks',
    'annoys',
    'helps',
    'reads',
    'chooses'
]

noun_b = [
    'the fence',
    'the shark',
    'the child',
    'the mistress',
    'the working class',
    'the book'
]

@display_helper
def generate_sentence():
    return choice(noun_a) + ' ' + choice(verb) + ' ' + choice(noun_b)

Button(description='Generate', style=ButtonStyle())

Output()

Kinda boring... We have two sets of nouns, could we combine them?

Well, nouns come in two flavors, proper and improper. Improper nouns have articles ("a/an" and "the"), while proper nouns don't (kind of). So we can have a set for proper nouns, and a set for improper nouns, and pick between them randomly

In [28]:
nouns_proper = [
    'John',
    'Jim',
    'Gordon Ramsay',
    'Steven',
    'Khan',
    'Roy',
    'Michael',
    'Vladmir Putin'
]

nouns_improper = [
    'fence',
    'shark',
    'child',
    'mistress',
    'citizen',
    'book',
    'egg'
]

verbs = [
    'jumps',
    'runs',
    'cooks',
    'annoys',
    'helps',
    'reads',
    'chooses',
    'writes',
    'punches'
]

def noun():
    if coin():
        return choice(nouns_proper)
    else:
        return 'the ' + choice(nouns_improper)

# Following the apparent convention
def verb():
    return choice(verbs)
    
@display_helper
def generate_sentence():
    return noun() + ' ' + verb() + ' ' + noun()

Button(description='Generate', style=ButtonStyle())

Output()

Now we got a bit more intrigue... the improper nouns only begin with "the", since there's a bit more code that needs to go into "a/an". Namely, we have to programatically choose between "a" and "an" depending on if the noun starts with a vowel.

Also, I'm going to imply that any noun that doesn't allow for both "a/an" and "the" is proper. This includes "the working class"

In [30]:
nouns_proper = [
    'John',
    'Jim',
    'Gordon Ramsay',
    'Steven',
    'Khan',
    'Roy',
    'Michael',
    'Vladmir Putin',
    'the working class',
    'the biker gang',
    'the sky',
    'France'
]

nouns_improper = [
    'fence',
    'shark',
    'child',
    'mistress',
    'citizen',
    'book',
    'egg'
]

verbs = [
    'jumps',
    'runs',
    'cooks',
    'annoys',
    'helps',
    'reads',
    'chooses',
    'writes',
    'punches'
]

def noun():
    if coin():
        return choice(nouns_proper)
    else:
        noun = choice(nouns_improper)
        if coin():
            return 'the ' + noun
        elif noun[0] in 'aeiou': # Usually not y
            return 'an ' + noun
        else:
            return 'a ' + noun
            

# Following the apparent convention
def verb():
    return choice(verbs)
    
@display_helper
def generate_sentence():
    return noun() + ' ' + verb() + ' ' + noun()

Button(description='Generate', style=ButtonStyle())

Output()